<a href="https://colab.research.google.com/github/a4ankan/MLP-architectures-on-the-MNIST-dataset/blob/master/Self_Driving_Car(6apr).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mounting Google Drive

#

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

!pip install pyunpack
!pip install patool

  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
  Stored in directory: /root/.cache/pip/wheels/41/22/19/af15ef6264c58b625a82641ed7483ad05e258fbd8925505227
Successfully built pyunpack easyprocess
    100% |████████████████████████████████| 81kB 6.9MB/s 


In [0]:
# Importing required libraries

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
import random
import cv2
import math
import numpy as np
import scipy
import scipy.misc
from scipy import pi
from subprocess import call
from datetime import datetime
from itertools import islice
import matplotlib.pyplot as plt 
import tensorflow as tf

In [0]:
# Creating a Directiory to store unpacked dataset

os.mkdir("Driving Data One")

In [0]:
from pyunpack import Archive
Archive('drive/My Drive/Autopilot-TensorFlow-master.rar').extractall('Driving Data One')

In [0]:
# Dataset Preparation

import scipy.misc
import random

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/data.txt") as f:
    for line in f:
        xs.append("Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


train_xs = xs[:int(len(xs) * 0.7)] # splitting data into 70:30 ratio, as per the task assigned
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


In [0]:
num_train_images = len(train_xs)
num_val_images = len(val_xs)

print(num_train_images)
print(num_val_images)
num_images = len(xs) # Total Number of Images

31784
13621


**Task Assigned**
1. Train Test Split - 70:30
2. Adam Optimizer Parameter - 1e-3 
3. Activation Function To be Used - Linear / Identity Function

In [0]:
# Model Architecture

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3], name="x")
y_ = tf.placeholder(tf.float32, shape=[None, 1],name="y_") # ground truth degree values

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32,name="keep_prob")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.matmul(h_fc4_drop, W_fc5) + b_fc5
y = tf.identity(y,name="predicted_steering_angle") # using an idetity function # predicted degree radians

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


**Training Model**

In [0]:
LOGDIR = 'linear'

# Lets start the tensorflow session
sess = tf.InteractiveSession()

In [0]:
L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver()

# op to write logs to Tensorboard
logs_path = 'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 1
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(num_images/batch_size)):
    xs, ys = LoadTrainBatch(batch_size)
    train_step.run(feed_dict={x: xs, y_: ys,keep_prob: 0.5})
    if i % 10 == 0:
      xs, ys = LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={x:xs, y_: ys,keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys,keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model1.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")
   

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0, Step: 0, Loss: 6.5272
Epoch: 0, Step: 10, Loss: 5.70684
Epoch: 0, Step: 20, Loss: 5.42571
Epoch: 0, Step: 30, Loss: 5.15121
Epoch: 0, Step: 40, Loss: 5.14422
Epoch: 0, Step: 50, Loss: 4.60156
Epoch: 0, Step: 60, Loss: 4.52047
Epoch: 0, Step: 70, Loss: 4.53373
Epoch: 0, Step: 80, Loss: 4.23739
Epoch: 0, Step: 90, Loss: 3.88578
Epoch: 0, Step: 100, Loss: 3.73123
Epoch: 0, Step: 110, Loss: 3.58113
Epoch: 0, Step: 120, Loss: 3.5014
Epoch: 0, Step: 130, Loss: 3.65248
Epoch: 0, Step: 140, Loss: 4.02363
Epoch: 0, Step: 150, Loss: 3.4782
Epoch: 0, Step: 160, Loss: 3.97738
Epoch: 0, Step: 170, Loss: 3.27448
Epoch: 0, Step: 180, Loss: 3.49371
Epoch: 0, Step: 190, Loss: 3.16173
Epoch: 0, Step: 200, Loss: 2.7864
Epoch: 0, Step: 210, Loss: 2.72281
Epoch: 0, Step: 220, Loss: 2.71062
Epoch: 0, Step: 230, Loss: 2.73927
Epoch: 0, Step: 240, Loss: 3.52635
Epoch: 0, Step: 250, Loss: 2.66391
Epoch: 0, Step: 260, Loss: 4.58

In [0]:
!pip3 install opencv-python

In [0]:
from glob import glob
images = glob('Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/*.jpg')
images

['Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/27829.jpg',
 'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/5198.jpg',
 'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/45237.jpg',
 'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/22734.jpg',
 'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/784.jpg',
 'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/31140.jpg',
 'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/26789.jpg',
 'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/26238.jpg',
 'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/28448.jpg',
 'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/d

In [0]:
(num_images)

45406

In [0]:
validation_images = images[:i]

In [0]:
len(validation_images)

31785

In [0]:
validation_images

In [0]:


import tensorflow as tf
import scipy.misc

import cv2
from subprocess import call
import math

sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph('linear/model2.ckpt.meta')

saver

saver.restore(sess, "linear/model2.ckpt")




graph = tf.get_default_graph()

y = graph.get_tensor_by_name("predicted_steering_angle:0")
original_image = graph.get_tensor_by_name("x:0")
keep_prob = graph.get_tensor_by_name("keep_prob:0")
#img = cv2.imread('steering_wheel_image.jpg',0)
#rows,cols = img.shape

#smoothed_angle = 0


# #read data.txt
xs = []
ys = []
with open("Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/data.txt") as f:
    for line in f:
        xs.append("Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.7)
print("Starting frameofvideo:" +str(i))
degree_results = []
for j in validation_images[:25000]:
    full_image = scipy.misc.imread(j, mode="RGB") 
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0

    degrees = y.eval(feed_dict={original_image: [image], keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
      #print("Predicted Steering angle: " + str(degrees))
      #full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    #image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    #degrees = y.eval(feed_dict={original_image: [image], keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
      #call("clear")
    print("Predicted Steering angle: " + str(degrees))
#     print("Steering angle: " + str(degrees) + " (pred)\t")
      #cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
      #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
      #and the predicted angle
      #smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
      #M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
      #dst = cv2.warpAffine(img,M,(cols,rows))
      #cv2.imshow("steering wheel", dst)
      #i += 1
    degree_results.append(degrees)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:Restoring parameters from linear/model2.ckpt
Starting frameofvideo:31785
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted S

In [0]:
L2NormConst = 0.001


train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver()

# op to write logs to Tensorboard
logs_path = 'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(num_images/batch_size)):
    xs, ys = LoadTrainBatch(batch_size)
    train_step.run(feed_dict={x: xs, y_: ys,keep_prob: 0.5})
    if i % 10 == 0:
      xs, ys = LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={x:xs, y_: ys,keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys,keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model2.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")
   

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0, Step: 0, Loss: 6.34417
Epoch: 0, Step: 10, Loss: 5.67376
Epoch: 0, Step: 20, Loss: 5.39657
Epoch: 0, Step: 30, Loss: 5.1088
Epoch: 0, Step: 40, Loss: 5.10385
Epoch: 0, Step: 50, Loss: 4.56497
Epoch: 0, Step: 60, Loss: 4.488
Epoch: 0, Step: 70, Loss: 4.4931
Epoch: 0, Step: 80, Loss: 4.19535
Epoch: 0, Step: 90, Loss: 3.8396
Epoch: 0, Step: 100, Loss: 3.6843
Epoch: 0, Step: 110, Loss: 3.52389
Epoch: 0, Step: 120, Loss: 3.43143
Epoch: 0, Step: 130, Loss: 3.61071
Epoch: 0, Step: 140, Loss: 3.98157
Epoch: 0, Step: 150, Loss: 3.42231
Epoch: 0, Step: 160, Loss: 3.94482
Epoch: 0, Step: 170, Loss: 3.21624
Epoch: 0, Step: 180, Loss: 3.46577
Epoch: 0, Step: 190, Loss: 3.12414
Epoch: 0, Step: 200, Loss: 2.72854
Epoch: 0, Step: 210, Loss: 2.65665
Epoch: 0, Step: 220, Loss: 2.63902
Epoch: 0, Step: 230, Loss: 2.67133
Epoch: 0, Step: 240, Loss: 3.49489
Epoch: 0, Step: 250, Loss: 2.60405
Epoch: 0, Step: 260, Loss: 5.2034

In [0]:


import tensorflow as tf
import scipy.misc

import cv2
from subprocess import call
import math

sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph('linear/model2.ckpt.meta')

saver

saver.restore(sess, "linear/model2.ckpt")




graph = tf.get_default_graph()

y = graph.get_tensor_by_name("predicted_steering_angle:0")
original_image = graph.get_tensor_by_name("x:0")
keep_prob = graph.get_tensor_by_name("keep_prob:0")
#img = cv2.imread('steering_wheel_image.jpg',0)
#rows,cols = img.shape

#smoothed_angle = 0


# #read data.txt
# xs = []
# ys = []
# with open("Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/data.txt") as f:
#     for line in f:
#         xs.append("Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/" + line.split()[0])
#         #the paper by Nvidia uses the inverse of the turning radius,
#         #but steering wheel angle is proportional to the inverse of turning radius
#         #so the steering wheel angle in radians is used as the output
#         ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images)
print("Starting frameofvideo:" +str(i))
degree_results = []
for j in images:
    full_image = scipy.misc.imread(j, mode="RGB") 
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0

    degrees = y.eval(feed_dict={original_image: [image], keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
      #print("Predicted Steering angle: " + str(degrees))
      #full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    #image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    #degrees = y.eval(feed_dict={original_image: [image], keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
      #call("clear")
    print("Predicted Steering angle: " + str(degrees))
#     print("Steering angle: " + str(degrees) + " (pred)\t")
      #cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
      #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
      #and the predicted angle
      #smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
      #M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
      #dst = cv2.warpAffine(img,M,(cols,rows))
      #cv2.imshow("steering wheel", dst)
      #i += 1
    degree_results.append(degrees)


INFO:tensorflow:Restoring parameters from linear/model2.ckpt


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Starting frameofvideo:100
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9

KeyboardInterrupt: ignored

In [0]:
len(degree_results)

In [0]:
# saver_model = tf.train.Saver()

sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph('linear/model1.ckpt.meta')

saver

In [0]:
saver.restore(sess, "linear/model1.ckpt")

graph = tf.get_default_graph()

y = graph.get_tensor_by_name("predicted_steering_angle:0")
original_image = graph.get_tensor_by_name("x:0")
keep_prob = graph.get_tensor_by_name("keep_prob:0")

# img = cv2.imread('Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/steering_wheel_image.jpg',0)
# rows,cols = img.shape

# smoothed_angle = 0


#read data.txt
xs = []
ys = []
with open("Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/data.txt") as f:
    for line in f:
        xs.append("Driving Data/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)
ys[:10]


#image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
# degrees = y.eval(feed_dict={original_image: [image], keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
#call("clear")
#print("Predicted Steering angle: " + str(degrees))
   
    

In [0]:
len(ys)

45406

In [0]:
i = math.ceil(num_images*0.7)
i

70

In [0]:
images[i]

'Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/32964.jpg'

In [0]:
len(images)

45568

In [0]:
for j in images[:i]:
  
  print(j)

In [0]:
degrees_results = []
i = math.ceil(num_images*0.7)
#print("Starting frame of video:" +str(i))
for j in images[:i]:
  full_image = scipy.misc.imread(j, mode="RGB") 
  image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0

  degrees = y.eval(feed_dict={original_image: [image], keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
  print("Predicted Steering angle: " + str(degrees))
#while(cv2.waitKey(10) != ord('q')):
  
  degrees_results.append(degrees)

#while(cv2.waitKey(10) != ord('q')):

Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted Steering angle: 1.9712261050812954
Predicted 

In [0]:
len(degrees_results)

25708

In [0]:
plt.imshow(full_image)

In [0]:
plt.imshow(image)

In [0]:
len(full_image)
type(full_image)


In [0]:
num_images

In [0]:
 a = scipy.misc.imread('Driving Data One/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/3124.jpg')

In [0]:
plt.imshow(a)

**MNIST Dataset Like Implementation, Code Copied from Garurav Sharma Notebook**

In [0]:
SAVEDIR = "New_Model"
sess = tf.InteractiveSession()
tf.reset_default_graph()
L2NormConst = 0.001


train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())

saver = tf.train.Saver()

epochs = 30
batch_size = 100
epoch_number, train_loss, test_loss,  = [], [], []

for epoch in range(epochs):
    train_avg_loss = 0
    test_avg_loss = 0
    te_loss_old = 10000  #any big number can be given
    
    for i in range(int(len(xs)/batch_size)):
        train_batch_x, train_batch_y = loadTrainBatch(batch_size)
        train_step.run(feed_dict = {x: train_batch_x, y_: train_batch_y, keep_prob: 0.5})
        tr_loss = loss.eval(feed_dict = {x: train_batch_x, y_: train_batch_y, keep_prob: 1.0})
        train_avg_loss += tr_loss / batch_size
    
        test_batch_x, test_batch_y = LoadValBatch(batch_size)
        te_loss_new = loss.eval(feed_dict = {x: test_batch_x, y_: test_batch_y, keep_prob: 1.0})
        test_avg_loss += te_loss_new / batch_size
        
        if te_loss_new < te_loss_old:
            print("Epoch: {}, Train_Loss: {}, Test_Loss: {} *".format(epoch+1, tr_loss, te_loss_new))
        else:
            print("Epoch: {}, Train_Loss: {}, Test_Loss: {}".format(epoch+1, tr_loss, te_loss_new))
        te_loss_old = te_loss_new
        
        if (i+1) % batch_size == 0:
            if not os.path.exists(SAVEDIR):
                os.makedirs(SAVEDIR)
            save_path = os.path.join(SAVEDIR, "model.ckpt")
            saver.save(sess = sess, save_path = save_path)
            print("Model saved at location {} at epoch {}".format(save_path, epoch + 1))
        
    epoch_number.append(epoch)
    train_loss.append(train_avg_loss)
    test_loss.append(test_avg_loss)
    
#creating dataframe and record all the losses and accuracies at each epoch
# log_frame = pd.DataFrame(columns = ["Epoch", "Train Loss", "Test Loss"])
# log_frame["Epoch"] = epoch_number
# log_frame["Train Loss"] = train_loss
# log_frame["Test Loss"] = test_loss
# log_frame.to_csv(os.path.join(SAVEDIR, "log.csv"), index = False)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


ValueError: ignored

In [0]:
x: xs, y_: ys,keep_prob: 0.5